In [2]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score
import numpy as np
import time

In [4]:
import EnsembleSelection

print(EnsembleSelection.SelectionStrategies.quality.value)

quality


In [3]:
n = 500
X, y = make_blobs(n_samples=n, centers=10, cluster_std=[i for i in range(10)], n_features=10)
from hdbscan import validity_index as dbcv_score

In [ ]:
dbcv_score(X, y)

In [75]:
y_pred = KMeans(n_clusters=10, n_init=1).fit_predict(X)

In [76]:
adjusted_rand_score(y, y_pred)
adjusted_mutual_info_score(y,y_pred)

0.7781736299685907

In [4]:

ensemble = []
start = time.time()
for k in range(2, 40):
    y_pred = KMeans(n_clusters=k, n_init=1).fit_predict(X)
    ensemble.append(y_pred)
ensemble_t = np.asarray(ensemble).transpose()
runtime_ensemble_gen = time.time() - start
runtime_ensemble_gen

/home/ubuntu/anaconda3/envs/automated_ensemble_clustering_39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ubuntu/anaconda3/envs/automated_ensemble_clustering_39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ubuntu/anaconda3/envs/automated_ensemble_clustering_39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ubuntu/anaconda3/envs/automated_ensemble_clustering_39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value 

13.735790729522705

In [5]:
ensemble_t.shape

(500, 38)

In [110]:
len(np.unique(ensemble_t[:, 37]))
len(np.unique(ensemble_t[:, 37]))

39

In [6]:
# Calculate pairwise NMI
# TODO: What if we do not use NMI, but instead the CVI? 
# I.e., we cluster the CVI values (but leave out CVI values with inf.!)

import time
start = time.time()
nmi_matrix = np.zeros((ensemble_t.shape[1], ensemble_t.shape[1]))
for i in range(ensemble_t.shape[1]):
    for j in range(ensemble_t.shape[1]):
        nmi_matrix[i,j] = adjusted_mutual_info_score(ensemble_t[:, i], ensemble_t[:, j])
runtime = time.time() - start
runtime

9.277326107025146

In [7]:
nmi_matrix

array([[1.        , 0.55397882, 0.45871328, ..., 0.25358855, 0.25204565,
        0.26114741],
       [0.55397882, 1.        , 0.70571575, ..., 0.39773724, 0.40040413,
        0.40595735],
       [0.45871328, 0.70571575, 1.        , ..., 0.48303691, 0.48344341,
        0.49271114],
       ...,
       [0.25358855, 0.39773724, 0.48303691, ..., 1.        , 0.73484406,
        0.74885121],
       [0.25204565, 0.40040413, 0.48344341, ..., 0.73484406, 1.        ,
        0.74434325],
       [0.26114741, 0.40595735, 0.49271114, ..., 0.74885121, 0.74434325,
        1.        ]])

In [8]:
# Cluster NMI matrix


m = 30
start = time.time()
nmi_clustering_result = KMeans(n_clusters=m).fit_predict(nmi_matrix)

selected_ensemble = np.zeros((n, m))

for cluster in np.unique(nmi_clustering_result):
    print(cluster)
    # get indices from ens_result 
    cluster_indices = np.where(cluster==nmi_clustering_result)[0]
    print(np.where(cluster==nmi_clustering_result)[0])
    print(nmi_matrix[np.where(cluster==nmi_clustering_result)])
    print(len(nmi_matrix[np.where(cluster==nmi_clustering_result)]))

    nmi_cluster = nmi_matrix[cluster_indices]
    print(np.mean(nmi_matrix[cluster_indices], axis=1))
    nmi_mean_instances = np.mean(nmi_cluster, axis=1)
    print(np.argmax(np.mean(nmi_matrix[cluster_indices], axis=1)))
    max_instance = np.argmax(nmi_mean_instances)
    print(cluster_indices[max_instance])
    ens_result_to_use = cluster_indices[max_instance]
    selected_ensemble[:, cluster] = ensemble_t[:, ens_result_to_use]
    #print(ens_result)

    #labeld_nmi_matrix
time.time()-start

/home/ubuntu/anaconda3/envs/automated_ensemble_clustering_39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0
[12 13 14 16]
[[0.31697691 0.47631012 0.58197888 0.66672645 0.68753973 0.77806404
  0.79692456 0.84579591 0.87116431 0.89055629 0.8685693  0.89745753
  1.         0.88948072 0.86899827 0.83872089 0.86921311 0.84421762
  0.81600682 0.82199077 0.82150698 0.83890396 0.80248238 0.79134368
  0.78983961 0.79844269 0.78540315 0.80727021 0.76354134 0.81665312
  0.76484332 0.78012564 0.76899457 0.75388222 0.77280622 0.73766891
  0.74237232 0.7486877 ]
 [0.31311815 0.47557304 0.58251316 0.65061804 0.68906226 0.77642281
  0.78706987 0.82708991 0.85330144 0.86116445 0.85417839 0.85137912
  0.88948072 1.         0.88297434 0.81511716 0.88456759 0.83999333
  0.83951747 0.82241307 0.80703414 0.83143088 0.80334606 0.77134302
  0.79433052 0.77537852 0.80388419 0.7918583  0.74009184 0.78372552
  0.73672815 0.78024913 0.74093192 0.74065117 0.75435449 0.73180428
  0.72808281 0.75376997]
 [0.29617162 0.46347421 0.57008066 0.64044119 0.6744412  0.75388018
  0.77497625 0.80903238 0.84551928 0.84124248 0.84

0.3383603096008301

In [124]:
len(selected_ensemble[:, -1])

10000

In [9]:
for i in range(selected_ensemble.shape[1]):
    print(len(np.unique(selected_ensemble[:, i])))

14
3
6
29
10
2
4
8
38
21
30
37
27
39
5
36
34
19
25
32
35
24
26
33
28
23
17
31
20
9
